In [ ]:
!pip install collinear

In [ ]:
from collinear.client import Client
from collinear.schemas.steer import SteerConfigInput
import logging

In [ ]:
OPENAI_API_KEY = ""
STEER_API_KEY = ""
client = Client(
    assistant_model_url="https://api.openai.com/v1",
    assistant_model_api_key=OPENAI_API_KEY,
    assistant_model_name="gpt-4o-mini",
    steer_api_key=STEER_API_KEY,
    timeout=120,
)
steer_config: SteerConfigInput = {
    "ages": ["young adult"],
    "genders": ["male"],
    "occupations": ["software engineer"],
    "intents": ["Cancel service"],
    "traits": {"confusion": [0, 1, 3]},
}

In [ ]:
simulations = client.simulate(
    steer_config=steer_config,
    k=1,
    num_exchanges=2,
    batch_delay=0.3,
)

# Pretty-print the simulated conversations and steer details.
for i, sim in enumerate(simulations, start=1):
    title = f"Conversation {i}"
    p = sim.steer
    if p is not None:
        print(
            "Steer Persona:\n"
            f"age={p.age}\n"
            f"gender={p.gender}\n"
            f"occupation={p.occupation}\n"
            f"intent={p.intent}\n"
            f"trait={p.trait}\n"
            f"intensity={p.intensity}"
        )
    print("---")
    for msg in sim.conv_prefix:
        role = str(msg.get("role", ""))
        content = str(msg.get("content", ""))
        if content:
            print(f"{role}: {content}")
    print(f"assistant: {sim.response}")
    print()
print("All simulations complete")

In [ ]:
result = client.assess(dataset=simulations)
print(f"Assessment: {result.message or '<no message>'}")

# Print the assessment for each piece of simulated data
for scores_map in result.evaluation_result:
    for scores in scores_map.values():
        print(f"  Score: {scores.score}")
        print(f"  Rationale: {scores.rationale}")